In [ ]:
# Note: include in the tests folder a .env file that contains the API keys for the services used in the tests    
import os
import pandas as pd  # type: ignore
import palimpzest as pz

if not os.environ.get('OPENAI_API_KEY'):
    from palimpzest.utils import load_env
    load_env()


### Columns definition
In the following cell we target schema of the case study we are interested in. Note how we are not specifying target attributes with fine grained metadata but rather with natural language, possibly ambiguous, specifications.

In [2]:
case_data_cols = [
    {"name": "case_submitter_id", "type": str, "desc": "The ID of the case"},
    {"name": "age_at_diagnosis", "type": int | float, "desc": "The age of the patient at the time of diagnosis"},
    {"name": "race", "type": str, "desc": "An arbitrary classification of a taxonomic group that is a division of a species."},
    {"name": "ethnicity", "type": str, "desc": "Whether an individual describes themselves as Hispanic or Latino or not."},
    {"name": "gender", "type": str, "desc": "Text designations that identify gender."},
    {"name": "vital_status", "type": str, "desc": "The vital status of the patient"},
    {"name": "ajcc_pathologic_t", "type": str, "desc": "Code of pathological T (primary tumor) to define the size or contiguous extension of the primary tumor (T), using staging criteria from the American Joint Committee on Cancer (AJCC)."},
    {"name": "ajcc_pathologic_n", "type": str, "desc": "The codes that represent the stage of cancer based on the nodes present (N stage) according to criteria based on multiple editions of the AJCC's Cancer Staging Manual."},
    {"name": "ajcc_pathologic_stage", "type": str, "desc": "The extent of a cancer, especially whether the disease has spread from the original site to other parts of the body based on AJCC staging criteria."},
    {"name": "tumor_grade", "type": int | float, "desc": "Numeric value to express the degree of abnormality of cancer cells, a measure of differentiation and aggressiveness."},
    {"name": "tumor_focality", "type": str, "desc": "The text term used to describe whether the patient's disease originated in a single location or multiple locations."},
    {"name": "tumor_largest_dimension_diameter", "type": int | float, "desc": "The tumor largest dimension diameter."},
    {"name": "primary_diagnosis", "type": str, "desc": "Text term used to describe the patient's histologic diagnosis, as described by the World Health Organization's (WHO) International Classification of Diseases for Oncology (ICD-O)."},
    {"name": "morphology", "type": str, "desc": "The Morphological code of the tumor, as described by the World Health Organization's (WHO) International Classification of Diseases for Oncology (ICD-O)."},
    {"name": "tissue_or_organ_of_origin", "type": str, "desc": "The text term used to describe the anatomic site of origin, of the patient's malignant disease, as described by the World Health Organization's (WHO) International Classification of Diseases for Oncology (ICD-O)."},
    {"name": "study", "type": str, "desc": "The last name of the author of the study, from the table name"}
]

file_cols = [
    {"name": "filename", "type": str, "desc": "The name of the file"},
    {"name": "contents", "type": bytes, "desc": "The contents of the file"}
]

xls_cols = file_cols + [
    {"name": "number_sheets", "type": int, "desc": "The number of sheets in the Excel file"},
    {"name": "sheet_names", "type": list[str], "desc": "The names of the sheets in the Excel file"},
]

table_cols = [
    {"name": "rows", "type": list[str], "desc": "The rows of the table"},
    {"name": "header", "type": list[str], "desc": "The header of the table"},
    {"name": "name", "type": str, "desc": "The name of the table"},
    {"name": "filename", "type": str, "desc": "The name of the file the table was extracted from"}
]

## Printing base dataset
In the following cell we print the base dataset we are interested in. Note how we are not specifying target attributes with fine grained metadata but rather with natural language, possibly ambiguous, specifications.

In [ ]:
def print_tables(output):
    for table in output:
        header = table.header
        subset_rows = table.rows[:3]

        print("Table name:", table.name)
        print(" | ".join(header)[:100], "...")
        for row in subset_rows:
            print(" | ".join(row)[:100], "...")
        print()


xls = pz.Dataset("testdata/biofabric-tiny")
patient_tables = xls.sem_add_columns(table_cols, cardinality=pz.Cardinality.ONE_TO_MANY)
output = patient_tables

policy = pz.MinCost()
config = pz.QueryProcessorConfig(
    policy=policy,
    cache=False,
    processing_strategy="no_sentinel",
)
data_record_collection = output.run(config)

print_tables(data_record_collection.data_records)

### Filtering stage
In the following cell we define the first part of the workload, that comprises a filtering stage responsible for selecting the tables from all the spreadsheets that contain relevant biometric information about the patient.


In [ ]:
xls = pz.Dataset("testdata/biofabric-tiny")
patient_tables = xls.sem_add_columns(table_cols, cardinality=pz.Cardinality.ONE_TO_MANY)
patient_tables = patient_tables.sem_filter("The table contains biometric information about the patient")

output = patient_tables

policy = pz.MinCost()
config = pz.QueryProcessorConfig(
    policy=policy,
    cache=False,
    processing_strategy="no_sentinel",
)
data_record_collection = patient_tables.run(config)

for table in data_record_collection:
    header = table.header
    subset_rows = table.rows[:3]

    print("Table name:", table.name)
    print(" | ".join(header)[:100], "...")
    for row in subset_rows:
        print(" | ".join(row)[:100], "...")
    print()

print(data_record_collection.execution_stats)

In [ ]:
policy = pz.MinCost()
config = pz.QueryProcessorConfig(
    policy=policy,
    cache=False,
    processing_strategy="no_sentinel",
    execution_strategy="parallel",
)
data_record_collection = patient_tables.run(config)

for table in data_record_collection:
    header = table.header
    subset_rows = table.rows[:3]

    print("Table name:", table.name)
    print(" | ".join(header)[:100], "...")
    for row in subset_rows:
        print(" | ".join(row)[:100], "...")
    print()

print(data_record_collection.execution_stats)

In [ ]:
print("Chosen plan:")
print(data_record_collection.executed_plans, "\n")
print("Stats:", data_record_collection.execution_stats)

## Matching stage
Once filtered, we can define the second part of the workload, that matches the patient records in the different tables and merging the information into a single table.

Minimum cost: GPT 3.5

In [ ]:
input_dataset = pz.Dataset("testdata/biofabric-tiny-filtered")
patient_tables = input_dataset.sem_add_columns(table_cols, cardinality=pz.Cardinality.ONE_TO_MANY)
case_data = patient_tables.sem_add_columns(case_data_cols, cardinality=pz.Cardinality.ONE_TO_MANY)

policy = pz.MinCost()
config = pz.QueryProcessorConfig(
    policy=policy,
    cache=False,
    allow_code_synth=False,
    processing_strategy="no_sentinel",
    execution_strategy="parallel",
)
data_record_collection = case_data.run(config) 

output_rows = []
for output_table in data_record_collection:
    output_rows.append(output_table.to_dict()) 

output_df = pd.DataFrame(output_rows)
display(output_df)

In [ ]:
print(data_record_collection.executed_plans, "\n")
print("Stats:", data_record_collection.execution_stats)

Maximum Quality: GPT 4

In [ ]:
input_dataset = pz.Dataset("testdata/biofabric-tiny-filtered")
patient_tables = input_dataset.sem_add_columns(table_cols, cardinality=pz.Cardinality.ONE_TO_MANY)
case_data = patient_tables.sem_add_columns(case_data_cols, cardinality=pz.Cardinality.ONE_TO_MANY)

policy = pz.MaxQuality()
config = pz.QueryProcessorConfig(
    policy=policy,
    cache=False,
    allow_code_synth=False,
    processing_strategy="no_sentinel",
    execution_strategy="parallel",
)
data_record_collection = case_data.run(config)

output_rows = []
for output_table in data_record_collection:
    output_rows.append(output_table.to_dict()) 

output_df = pd.DataFrame(output_rows)
display(output_df)

In [ ]:
print(data_record_collection.executed_plans, "\n")
print("Stats:", data_record_collection.execution_stats)

## End to end

Minimum cost : GPT 3.5

In [14]:
xls = pz.Dataset("testdata/biofabric-tiny")
patient_tables = xls.sem_add_columns(table_cols, cardinality=pz.Cardinality.ONE_TO_MANY)
patient_tables = patient_tables.sem_filter("The table contains biometric information about the patient")
case_data = patient_tables.sem_add_columns(case_data_cols, cardinality=pz.Cardinality.ONE_TO_MANY)

policy = pz.MinCost()
config = pz.QueryProcessorConfig(
    policy=policy,
    cache=False,
    allow_code_synth=False,
    allow_token_reduction=False,
    processing_strategy="streaming",
    execution_strategy="sequential",
)
iterable = case_data.run(config)

output_rows = []
for data_record_collection in iterable:  # noqa: B007
    for output_table in data_record_collection:
        print(output_table.to_dict().keys())
        output_rows.append(output_table.to_dict()) 

output_df = pd.DataFrame(output_rows)
display(output_df)

Available models:  [GPT_3_5, GPT_4]
LOGICAL PLANS: 2
INITIAL PLANS: 6
INITIAL PLANS: 4
DEDUP PLANS: 8
PARETO PLANS: 1
Time for planning: 0.02268195152282715
Iteration number:  1 Last record:  False
dict_keys(['rows', 'filename', 'header', 'name', 'age_at_diagnosis', 'ajcc_pathologic_n', 'ajcc_pathologic_stage', 'ajcc_pathologic_t', 'case_submitter_id', 'ethnicity', 'gender', 'morphology', 'primary_diagnosis', 'race', 'study', 'tissue_or_organ_of_origin', 'tumor_focality', 'tumor_grade', 'tumor_largest_dimension_diameter', 'vital_status'])
Iteration number:  2 Last record:  False
Iteration number:  3 Last record:  True


,rows,filename,header,name,age_at_diagnosis,ajcc_pathologic_n,ajcc_pathologic_stage,ajcc_pathologic_t,case_submitter_id,ethnicity,gender,morphology,primary_diagnosis,race,study,tissue_or_organ_of_origin,tumor_focality,tumor_grade,tumor_largest_dimension_diameter,vital_status
0,"[[S001, C3L-00006, No, 2, 5, 128N, C3L-00006-0...",dou_mmc1.xlsx,"[idx, Proteomics_Participant_ID, Case_excluded...",dou_mmc1.xlsx_UCEC_CPTAC3_meta_table_V2.1,Age of the patient at the time of diagnosis,The codes that represent the stage of cancer b...,"The extent of a cancer, especially whether the...",Path_Stage_Primary_Tumor-pT,Proteomics_Participant_ID,Whether an individual describes themselves as ...,Text designations that identify gender,ICD-O Morphological code,Text term used to describe the patient's histo...,An arbitrary classification of a taxonomic gro...,CPTAC3,The text term used to describe the anatomic si...,The text term used to describe whether the pat...,Numeric value to express the degree of abnorma...,value,The vital status of the patient


In [ ]:
output_table.__dict__.keys()

Maximum Cost: GPT 4

In [18]:
xls = pz.Dataset("testdata/biofabric-tiny")
patient_tables = xls.sem_add_columns(table_cols, cardinality=pz.Cardinality.ONE_TO_MANY)
patient_tables = patient_tables.sem_filter("The table contains biometric information about the patient")
case_data = patient_tables.sem_add_columns(case_data_cols, cardinality=pz.Cardinality.ONE_TO_MANY)

policy = pz.MaxQuality()
config = pz.QueryProcessorConfig(
    policy=policy,
    cache=False,
    allow_token_reduction=False,
    allow_code_synth=False,
    processing_strategy="streaming",
    execution_strategy="sequential",
)
iterable = case_data.run(config)

output_rows = []
for data_record_collection in iterable:  # noqa: B007
    for output_table in data_record_collection:
        print(output_table.to_dict().keys())
        output_rows.append(output_table.to_dict()) 

output_df = pd.DataFrame(output_rows)
display(output_df)

LOGICAL PLANS: 1
INITIAL PLANS: 27
DEDUP PLANS: 27
PARETO PLANS: 17
----------
Policy is: Maximum Quality
Chosen plan: Time est: 411.647 -- Cost est: 0.973 -- Quality est: 0.486
 0. MarshalAndScanDataOp -> File 

 1. File -> InduceFromCandidateOp -> XLSFile 
    Using Model.GPT_3_5
    (contents,filena...) -> (contents,filena...)

 2. XLSFile -> InduceFromCandidateOp -> Table 
    Using Model.GPT_4
    (contents,filena...) -> (filename,header...)

 3. Table -> FilterCandidateOp -> Table 
    Using Model.GPT_4
    Filter: "The table contains biometric information about the patient"
    (filename,header...) -> (filename,header...)

 4. Table -> InduceFromCandidateOp -> CaseData 
    Using Model.GPT_4
    (filename,header...) -> (age_at_diagnosi...)

Bonded query processing error: No output objects were generated with bonded query - trying with conventional query...
BondedQuery Error: No output objects were generated with bonded query - trying with conventional query...
Falling back to co

,age_at_diagnosis,ajcc_pathologic_n,ajcc_pathologic_stage,ajcc_pathologic_t,case_submitter_id,ethnicity,gender,morphology,primary_diagnosis,race,study,tissue_or_organ_of_origin,tumor_focality,tumor_grade,tumor_largest_dimension_diameter,vital_status
0,64.0,pN0,Stage I,pT1a (FIGO IA),C3L-00006,Not-Hispanic or Latino,Female,Endometrioid,FIGO grade 1,White,UCEC_CPTAC3,Anterior endometrium,Unifocal,Cannot be determined,2.9,No
1,58.0,pNX,Stage IV,pT1a (FIGO IA),C3L-00008,Not-Hispanic or Latino,Female,Endometrioid,FIGO grade 1,White,UCEC_CPTAC3,Posterior endometrium,Unifocal,Cannot be determined,3.5,No
2,50.0,pN0,Stage I,pT1a (FIGO IA),C3L-00032,Not-Hispanic or Latino,Female,Endometrioid,FIGO grade 2,White,UCEC_CPTAC3,"Other, specify",Unifocal,Cannot be determined,4.5,Yes
3,NaN,None,None,None,C3L-00084,None,None,Carcinosarcoma,None,None,UCEC_CPTAC3,None,None,None,NaN,None
4,75.0,pNX,Stage I,pT1a (FIGO IA),C3L-00090,Not-Hispanic or Latino,Female,Endometrioid,FIGO grade 2,White,UCEC_CPTAC3,"Other, specify",Unifocal,Cannot be determined,3.5,No
5,729.0,N2b,Stage III,T4a,01CO001,None,Male,Mucinous,None,None,Vasaikar,Sigmoid Colon,None,None,NaN,Living
6,838.0,N0,Stage II,T3,01CO005,None,Female,Not Mucinous,None,None,Vasaikar,Sigmoid Colon,None,None,NaN,Deceased
7,904.0,N2b,Stage III,T4a,01CO006,None,Female,Mucinous,None,None,Vasaikar,Ascending Colon,None,None,NaN,Living
8,652.0,N0,Stage II,T3,01CO008,None,Female,Mucinous,None,None,Vasaikar,Descending Colon,None,None,NaN,Living
9,58.0,None,None,None,C3L-00104,Not-Hispanic or Latino,Male,None,None,White,wang,Frontal Lobe,None,None,NaN,Deceased
